In [9]:
import sys
sys.path.append('..')
import processing
import configs.common as cc
import train
from train import new_model
import torch
import configs.paths as paths
import argparse
import os
from collections import Counter
from generate import generate

data_root = "/home/s203861/midi-classical-music/np_data/data"
model_name = "transformer"

if model_name == "mamba":
    model = new_model("mamba")
    model.load_state_dict(torch.load(cc.config.models.mamba))
    model.to('cuda')
if model_name == "xlstm":
    model = new_model("xlstm")
    model.load_state_dict(torch.load(cc.config.models.xlstm))
    model.to('cuda')
if model_name == "transformer":
    model = new_model("transformer")
    model.load_state_dict(torch.load(cc.config.models.transformer))
    model.to('cuda')

In [10]:
import time

loader = processing.DatasetLoader(data_root)
dataloader = loader.get_dataloader_full()

num_outputs = 100
inference_times = []
model.eval()

idx = 0
for src, trg, meta in dataloader:
    start_time = time.time()
    
    for _ in range(num_outputs):
        with torch.no_grad():
            model(src, meta)
        idx += 1
    
    end_time = time.time()
    
    total_time = end_time - start_time
    avg_time_per_output = total_time / num_outputs
    
    inference_times.append(avg_time_per_output)
    
    break

# Print the average inference time for 100 outputs
print(f"Average inference time per output: {avg_time_per_output:.6f} seconds: total time {total_time}")

Average inference time per output: 0.038017 seconds: total time 3.8016529083251953


In [3]:
torch.cuda.memory_allocated()

414781952

In [8]:
from mamba_ssm.utils.generation import InferenceParams

model._decoding_cache.inference_params = InferenceParams(
    max_seqlen=cc.config.values.block_len+6,
    max_batch_size=len(src),
    seqlen_offset=1
)

for layer in self.layers:
    conv_state, ssm_state = layer.allocate_inference_cache(batch_size=len(src), max_seqlen=cc.config.values.block_len+6)
    inference_params.key_value_memory_dict[layer.layer_idx] = (conv_state, ssm_state)


AttributeError: 'NoneType' object has no attribute 'inference_params'

In [20]:
from mamba_ssm.utils.generation import decode
loader = processing.DatasetLoader(data_root)
dataloader = loader.get_dataloader_full()
for src, trg, meta in dataloader:
    break
decode(src,model,1024)

TypeError: Mamba2.forward() got an unexpected keyword argument 'position_ids'

In [19]:
from mamba_ssm import Mamba2
from mamba_ssm.utils.generation import InferenceParams

model = Mamba2(
    d_model=1024, 
    d_state=64,  # SSM state expansion factor, typically 64 or 128
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")
model._decoding_cache = type("Cache", (), {})()
model._decoding_cache.inference_params = InferenceParams(
            max_seqlen=cc.config.values.block_len+6,
            max_batch_size=len(src),
            seqlen_offset=1
        )